# Preparando o ambiente

⚠Download Docker - Windows

- ⚠[Docker](https://docs.docker.com/get-docker/)
- ⚠[Docker Compose](https://docs.docker.com/compose/install/)

⚠Instalação Docker - Windows

- [Link de Atualização Windows 10 (20.04)](https://www.microsoft.com/pt-br/software-download/windows10)
- [Link para instalação do Docker Desktop no **Windows**](https://hub.docker.com/editions/community/docker-ce-desktop-windows)
- [Download WSL 2](https://wslstorestorage.blob.core.windows.net/wslblob/wsl_update_x64.msi)

#### Pelo terminal do WSL2, crie um diretório para o projeto
$ mkdir projeto-final-spark

#### Acesse o diretório
$cd projeto-final-spark


#### No diretório spark, clone o projeto: 
$ git clone https://github.com/rodrigo-reboucas/docker-bigdata.git spark

#### Confirme se o arquivo foi baixado
$ ls


#### Entra na pasta spark e baixe as imagens: 

$ cd spark

$ docker-compose -f docker-compose-parcial.yml pull

#### Verifique se as imagens estão sendo listadas
$ docker image ls


------

# Enviar os dados para o HDFS

#### Pelo terminal do WSL2, dentro da pasta spark, baixe o arquivo de dados .rar dentro da pasta spark:
$ curl -O https://mobileapps.saude.gov.br/esus-vepi/files/unAFkcaNDeXajurGB7LChj8SgQYS2ptm/04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar

#### Instale o unrar para descompactar o arquivo .rar 
$ sudo apt install unrar

#### Descompacte o arquivo .rar 
$ unrar x 04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar

#### Crie um diretório input dentro da pasta spark
$ mkdir input

#### Enviar os arquivos .csv para a pasta input
$ sudo mv *.csv /home/cicero/projeto-final-spark/spark/input

#### Inicie todos os serviços
$ docker-compose -f docker-compose-parcial.yml up -d

#### Entre no namenode
$ docker exec -it namenode bash

#### Crie a pasta projeto-final-spark no HDFS para salvar os arquivos de dados .CSV
$ hdfs dfs -mkdir -p /user/cicero/projeto-final-spark

#### Envie oo arquivos de dados .CSV para a pasta projeto-final-spark no HDFS
$ hdfs dfs -put /input/*.csv /user/cicero/projeto-final-spark

#### Confirme se os arquivos foram enviados
$ hdfs dfs -ls /user/cicero/projeto-final-spark

------
#### Pelo navegador acesso o jupyter-notebook
$ http://localhost:8889/

##### ⚠ Obs.: Depois crie um arquivo chamado projeto_final_spark_nivel_basico, este que seguiremos com os próximos passos de agora em diante.
------

# Otimizar todos os dados do hdfs para uma tabela Hive particionada por município

In [66]:
#Importe as bibliotecas necessárias
from pyspark.sql import *
from pyspark.sql.types import *

In [67]:
#Acesse os arquivos pelo HDFS
!hdfs dfs -ls "hdfs:///user/cicero/projeto-final-spark/"

Found 4 items
-rw-r--r--   3 root supergroup   62492959 2021-11-08 22:43 hdfs:///user/cicero/projeto-final-spark/HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv
-rw-r--r--   3 root supergroup   76520681 2021-11-08 22:43 hdfs:///user/cicero/projeto-final-spark/HIST_PAINEL_COVIDBR_2020_Parte2_06jul2021.csv
-rw-r--r--   3 root supergroup   91120916 2021-11-08 22:43 hdfs:///user/cicero/projeto-final-spark/HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv
-rw-r--r--   3 root supergroup    3046774 2021-11-08 22:43 hdfs:///user/cicero/projeto-final-spark/HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv


In [68]:
#Acessar os arquivos e mostrar a primeira linha 
rdd = sc.textFile("hdfs:///user/cicero/projeto-final-spark/")
rdd.first()

'regiao;estado;municipio;coduf;codmun;codRegiaoSaude;nomeRegiaoSaude;data;semanaEpi;populacaoTCU2019;casosAcumulado;casosNovos;obitosAcumulado;obitosNovos;Recuperadosnovos;emAcompanhamentoNovos;interior/metropolitana'

In [69]:
#Mostrar as primieras linhas para entender os dados
rdd.take(200)

['regiao;estado;municipio;coduf;codmun;codRegiaoSaude;nomeRegiaoSaude;data;semanaEpi;populacaoTCU2019;casosAcumulado;casosNovos;obitosAcumulado;obitosNovos;Recuperadosnovos;emAcompanhamentoNovos;interior/metropolitana',
 'Brasil;;;76;;;;2020-02-25;9;210147125;0;0;0;0;;;',
 'Brasil;;;76;;;;2020-02-26;9;210147125;1;1;0;0;;;',
 'Brasil;;;76;;;;2020-02-27;9;210147125;1;0;0;0;;;',
 'Brasil;;;76;;;;2020-02-28;9;210147125;1;0;0;0;;;',
 'Brasil;;;76;;;;2020-02-29;9;210147125;2;1;0;0;;;',
 'Brasil;;;76;;;;2020-03-01;10;210147125;2;0;0;0;;;',
 'Brasil;;;76;;;;2020-03-02;10;210147125;2;0;0;0;;;',
 'Brasil;;;76;;;;2020-03-03;10;210147125;2;0;0;0;;;',
 'Brasil;;;76;;;;2020-03-04;10;210147125;3;1;0;0;;;',
 'Brasil;;;76;;;;2020-03-05;10;210147125;7;4;0;0;;;',
 'Brasil;;;76;;;;2020-03-06;10;210147125;13;6;0;0;;;',
 'Brasil;;;76;;;;2020-03-07;10;210147125;19;6;0;0;;;',
 'Brasil;;;76;;;;2020-03-08;11;210147125;25;6;0;0;;;',
 'Brasil;;;76;;;;2020-03-09;11;210147125;25;0;0;0;;;',
 'Brasil;;;76;;;;2020-03-

In [79]:
'''
#Criar um Data Frame com Schema organizado.
colunas_painel_covidbr = [ StructField("regiao", StringType()),
                          StructField("estado", StringType()),
                          StructField("municipio", StringType()),
                          StructField("coduf", StringType()),
                          StructField("codmun", StringType()),
                          StructField("codRegiaoSaude", StringType()),
                          StructField("nomeRegiaoSaude", StringType()),
                          StructField("data", DateType()),
                          StructField("semanaEpi", StringType()),
                          StructField("populacaoTCU2019", IntegerType()),
                          StructField("casosAcumulado", IntegerType()),
                          StructField("casosNovos", IntegerType()),
                          StructField("obitosAcumulado", IntegerType()),
                          StructField("obitosNovos", IntegerType()),
                          StructField("Recuperadosnovos", IntegerType()),
                          StructField("emAcompanhamentoNovos", IntegerType()),
                          StructField("interior/metropolitana", IntegerType())]

schema = StructType(colunas_painel_covidbr)
df_painel_covidbr = spark.read.option("header","true").option("delimiter",";").csv("hdfs:///user/cicero/projeto-final-spark/")
'''

#Utilizando o option("inferSchema","true") para que automaticamente o spark identifique os tipos de dados das colunas
df_painel_covidbr = spark.read.option("sep",";").option("header","true").option("inferSchema","true").csv("hdfs:///user/cicero/projeto-final-spark/")

In [77]:
#Visualize o schema
df_painel_covidbr.printSchema()
df_painel_covidbr.head()

root
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- coduf: integer (nullable = true)
 |-- codmun: integer (nullable = true)
 |-- codRegiaoSaude: integer (nullable = true)
 |-- nomeRegiaoSaude: string (nullable = true)
 |-- data: timestamp (nullable = true)
 |-- semanaEpi: integer (nullable = true)
 |-- populacaoTCU2019: integer (nullable = true)
 |-- casosAcumulado: decimal(10,0) (nullable = true)
 |-- casosNovos: integer (nullable = true)
 |-- obitosAcumulado: integer (nullable = true)
 |-- obitosNovos: integer (nullable = true)
 |-- Recuperadosnovos: integer (nullable = true)
 |-- emAcompanhamentoNovos: integer (nullable = true)
 |-- interior/metropolitana: integer (nullable = true)



Row(regiao='Brasil', estado=None, municipio=None, coduf=76, codmun=None, codRegiaoSaude=None, nomeRegiaoSaude=None, data=datetime.datetime(2020, 2, 25, 0, 0), semanaEpi=9, populacaoTCU2019=210147125, casosAcumulado=Decimal('0'), casosNovos=0, obitosAcumulado=0, obitosNovos=0, Recuperadosnovos=None, emAcompanhamentoNovos=None, interior/metropolitana=None)

In [83]:
df_painel_covidbr.show(5, vertical=True)

-RECORD 0-------------------------------------
 regiao                 | Brasil              
 estado                 | null                
 municipio              | null                
 coduf                  | 76                  
 codmun                 | null                
 codRegiaoSaude         | null                
 nomeRegiaoSaude        | null                
 data                   | 2020-02-25 00:00:00 
 semanaEpi              | 9                   
 populacaoTCU2019       | 210147125           
 casosAcumulado         | 0                   
 casosNovos             | 0                   
 obitosAcumulado        | 0                   
 obitosNovos            | 0                   
 Recuperadosnovos       | null                
 emAcompanhamentoNovos  | null                
 interior/metropolitana | null                
-RECORD 1-------------------------------------
 regiao                 | Brasil              
 estado                 | null                
 municipio   

In [ ]:
#Inserindo os dados no Apache Hive


In [ ]:
#Separando os dados por municípios


In [ ]:
#Salvando os dados no Hive


# Criar as 3 visualizações pelo Spark com os dados enviados para o HDFS 

# Salvar a primeira visualização como tabela Hive

# Salvar a segunda visualização com formato parquet e compressão snappy

# Salvar a terceira visualização em um tópico no Kafka

# Criar a visualização pelo Spark com os dados enviados para o HDFS